In [1]:
%pip install ultralytics kagglehub wandb albumentations opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
!nvidia-smi

Sun Oct  5 14:29:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
import shutil
from tqdm import tqdm

base_dir = "/kaggle/input/ppe-detection"

# ต้นทาง
train_dir = os.path.join(base_dir, "train_aug")
valid_dir = os.path.join(base_dir, "valid")

output_dir = "/kaggle/working/train_valid"
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

def copy_with_prefix(src_dir, dst_dir, prefix):
    for subfolder in ["images", "labels"]:
        src_path = os.path.join(src_dir, subfolder)
        dst_path = os.path.join(dst_dir, subfolder)
        os.makedirs(dst_path, exist_ok=True)
        for file in tqdm(os.listdir(src_path), desc=f"Copying {prefix} {subfolder}"):
            src_file = os.path.join(src_path, file)
            new_name = f"{prefix}_{file}"
            dst_file = os.path.join(dst_path, new_name)
            shutil.copy(src_file, dst_file)

# คัดลอก train_aug และ valid โดยเติม prefix
copy_with_prefix(train_dir, output_dir, "train")
copy_with_prefix(valid_dir, output_dir, "valid")

print("✅ output dir ->", output_dir)


Copying valid labels: 100%|██████████| 8814/8814 [01:47<00:00, 81.83it/s]

✅ output dir -> /kaggle/working/train_valid


In [4]:
!rm -rf /kaggle/working/kfold_datasets

In [5]:
import os
from tqdm import tqdm
from sklearn.model_selection import KFold
from glob import glob

base_dir = "/kaggle/working/train_valid"
image_paths = sorted(glob(os.path.join(base_dir, "images", "*.jpg")))
label_paths = [p.replace("images", "labels").replace(".jpg", ".txt") for p in image_paths]

output_base = "/kaggle/working/kfold_symlinks"
os.makedirs(output_base, exist_ok=True)

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(image_paths), 1):
    fold_dir = os.path.join(output_base, f"fold_{fold}")
    for sub in ["train/images", "train/labels", "val/images", "val/labels"]:
        os.makedirs(os.path.join(fold_dir, sub), exist_ok=True)

    for i in tqdm(train_idx, desc=f"Fold {fold} train"):
        os.symlink(image_paths[i], os.path.join(fold_dir, "train/images", os.path.basename(image_paths[i])))
        os.symlink(label_paths[i], os.path.join(fold_dir, "train/labels", os.path.basename(label_paths[i])))

    for i in tqdm(val_idx, desc=f"Fold {fold} val"):
        os.symlink(image_paths[i], os.path.join(fold_dir, "val/images", os.path.basename(image_paths[i])))
        os.symlink(label_paths[i], os.path.join(fold_dir, "val/labels", os.path.basename(label_paths[i])))

print("✅ Created K-Fold dataset using symlinks (no extra disk space used):", output_base)


Fold 5 val: 100%|██████████| 7915/7915 [00:00<00:00, 26618.00it/s]

✅ Created K-Fold dataset using symlinks (no extra disk space used): /kaggle/working/kfold_symlinks


In [6]:
import yaml
import os

classes = [
        "Gloves",
        "Goggles",
        "Hardhat",
        "Mask",
        "NO-Gloves",
        "NO-Goggles",
        "NO-Hardhat",
        "NO-Mask",
        "NO-Safety Vest",
        "Person",
        "Safety Vest",
    ]

output_base = "/kaggle/working/kfold_symlinks"

for fold in range(1, 6):
    data_yaml = {
        "train": f"{output_base}/fold_{fold}/train/images",
        "val": f"{output_base}/fold_{fold}/val/images",
        "nc": len(classes),
        "names": classes,
    }

    yaml_path = os.path.join(output_base, f"fold_{fold}_data.yaml")
    with open(yaml_path, "w") as f:
        yaml.dump(data_yaml, f)

    print(f"✅ created yaml: {yaml_path} finished")


✅ created yaml: /kaggle/working/kfold_symlinks/fold_1_data.yaml finished
✅ created yaml: /kaggle/working/kfold_symlinks/fold_2_data.yaml finished
✅ created yaml: /kaggle/working/kfold_symlinks/fold_3_data.yaml finished
✅ created yaml: /kaggle/working/kfold_symlinks/fold_4_data.yaml finished
✅ created yaml: /kaggle/working/kfold_symlinks/fold_5_data.yaml finished


In [7]:
!yolo settings wandb=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Updated 'wandb=True'
JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/kaggle/working/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "1bfc3e992d24318da58ddee183be5bf9388a31f26bab1738e986ec4d297417ff",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [8]:
import wandb
import os
os.environ['WANDB_API_KEY'] = 'c7c1481bd47886cb1e195b3c538df89af9a716cd'

wandb.login()

wandb.init(project="PPE-Detection-KFold", name="yolov8_run", job_type="train")

wandb: Currently logged in as: kitton to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251005_144837-sg6wslgw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run yolov8_run
wandb: ⭐️ View project at https://wandb.ai/kitton/PPE-Detection-KFold
wandb: 🚀 View run at https://wandb.ai/kitton/PPE-Detection-KFold/runs/sg6wslgw


In [9]:
from ultralytics import YOLO

model_path = "yolov8n.pt"

yaml_base = "/kaggle/working/kfold_symlinks" 
fold_yamls = [f"{yaml_base}/fold_{i}_data.yaml" for i in range(1, 6)]

for fold, yaml_path in enumerate(fold_yamls, 1):
    print(f"=== Training fold {fold} ===")
    model = YOLO(model_path)

    model.train(
        data=yaml_path,
        epochs=10,
        imgsz=640,
        batch=16,
        name=f"fold_{fold}_exp",
        project="K-Fold-Training",
        exist_ok=True
    )


=== Training fold 1 ===
Ultralytics 8.3.205 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/kfold_symlinks/fold_1_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold_1_exp, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       7916      15896      0.618      0.559      0.551      0.302
                Gloves        385        837      0.711      0.784      0.783      0.398
               Goggles        631        691      0.662      0.716      0.742      0.383
               Hardhat       2723       7462      0.684      0.725       0.72      0.381
                  Mask        214        413      0.416      0.579      0.339      0.196
             NO-Gloves        485       1057       0.59       0.47        0.5      0.214
            NO-Goggles        592        754      0.678      0.584      0.668      0.348
            NO-Hardhat        933       2414      0.597      0.652      0.621      0.366
               NO-Mask        289        437      0.438      0.451      0.308      0.182
        NO-Safety Vest        173        349      0.713    0.00732     0.0879     0.0456
                Person        174        265       0.76      0.808      0.855      0.547
           Safety Ves

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆█▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▃▆█▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▃▆█▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▁▂▃▅▅▆▇▇██
wandb:     metrics/mAP50-95(B) ▁▂▃▄▅▆▇▇██
wandb:    metrics/precision(B) ▃▁▄▄▆▇▇███
wandb:       metrics/recall(B) ▁▂▃▄▅▆▆▇██
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▅▄▃▃▂▂▁▁
wandb:          train/cls_loss █▄▃▃▂▂▂▁▁▁
wandb:          train/dfl_loss █▆▆▅▄▃▂▂▁▁
wandb:            val/box_loss █▇▆▅▄▃▂▂▁▁
wandb:            val/cls_loss █▇▇▃▃▃▂▂▂▁
wandb:            val/dfl_loss █▇▇▅▄▃▂▂▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 7e-05
wandb:                  lr/pg1 7e-05
wandb:                  lr/pg2 7e-05
wandb:        metrics/mAP50(B) 0.55118
wandb:     metrics/mAP50-95(B) 0.302
wandb:    metrics/precisi

=== Training fold 2 ===
Ultralytics 8.3.205 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/kfold_symlinks/fold_2_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fold_2_exp, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

...

wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251005_190010-8wxduzow
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fold_5_exp
wandb: ⭐️ View project at https://wandb.ai/kitton/K-Fold-Training
wandb: 🚀 View run at https://wandb.ai/kitton/K-Fold-Training/runs/8wxduzow


Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       7915      15871      0.658      0.571      0.571      0.313
                Gloves        367        798      0.708      0.822      0.788      0.388
               Goggles        710        780      0.712      0.745      0.775      0.415
               Hardhat       2729       7383      0.687      0.729       0.72      0.379
                  Mask        284        540      0.498      0.529      0.401      0.239
             NO-Gloves        492       1047      0.656      0.481      0.551       0.24
            NO-Goggles        573        721      0.679      0.641      0.716      0.375
            NO-Hardhat        891       2341      0.565      0.625      0.577      0.333
               NO-Mask        300        412      0.503       0.49       0.42      0.254
        NO-Safety Vest        186        383          1          0     0.0811     0.0439
                Person        177        253      0.707       0.81      0.814      0.515
           Safety Ves

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆█▇▆▅▄▃▂▁
wandb:                  lr/pg1 ▃▆█▇▆▅▄▃▂▁
wandb:                  lr/pg2 ▃▆█▇▆▅▄▃▂▁
wandb:        metrics/mAP50(B) ▁▃▄▅▆▆▇▇██
wandb:     metrics/mAP50-95(B) ▁▃▃▅▅▆▇▇██
wandb:    metrics/precision(B) ▁▃▄▅▆▆▇▇██
wandb:       metrics/recall(B) ▁▃▃▄▅▆▆▇██
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▆▅▄▄▃▂▂▁▁
wandb:          train/cls_loss █▄▃▃▂▂▂▁▁▁
wandb:          train/dfl_loss █▆▅▅▄▃▃▂▁▁
wandb:            val/box_loss █▇▆▄▄▃▂▂▁▁
wandb:            val/cls_loss █▄▄▂▃▂▂▂▁▁
wandb:            val/dfl_loss █▇▆▅▃▃▂▂▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 7e-05
wandb:                  lr/pg1 7e-05
wandb:                  lr/pg2 7e-05
wandb:        metrics/mAP50(B) 0.57064
wandb:     metrics/mAP50-95(B) 0.31293
wandb:    metrics/preci